In [2]:
# ------------------------------
# Mount Google Drive
# ------------------------------
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Install dependencies
!pip install simpletransformers
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 137.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=bc039d036bf7dbc93e238b2c916daf6a1ad093a9e4c92e6a77cae5a5a7994257
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
# Load training dataset
import pandas as pd


In [5]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

file_path = '/content/drive/MyDrive/meme_classifications/ocr_text/train/STask_B_train.csv'
combined_df = pd.read_csv(file_path)

print("Training Data:")
print(combined_df.head())

# Ensure labels are integers
combined_df['label'] = combined_df['label'].astype(int)
print("Label distribution:")
print(combined_df['label'].value_counts())


Training Data:
       index                                               text  label
0  10016.png   cope mald seeth p trans rights made with mematic    0.0
1  10017.png  restrict access to birth control overturning r...    3.0
2  10024.png  09 54 1 search 30 mins people are condemning t...    2.0
3  10027.png  milliebobbybrown 2h i just stole this car from...    2.0
4  10030.png          me trying to live my life lgbt propaganda    2.0
Label distribution:
label
2    931
0    617
3    238
1    199
Name: count, dtype: int64


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Load the dataset (replace 'data.csv' with your actual dataset file)
data = combined_df

# Preprocess the text data
max_words = 1000  # Maximum number of words to keep in the vocabulary
max_sequence_length = 100  # Maximum length of each text sequence

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['text'])

sequences = tokenizer.texts_to_sequences(data['text'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# Pad sequences to have the same length
data_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
labels = data['label'].values

# Convert labels to one-hot encoded format
one_hot_labels = pd.get_dummies(labels)

train_data, test_data, train_labels, test_labels = train_test_split(
    data_padded, one_hot_labels, test_size=0.2, random_state=42
)

Found 12830 unique tokens.


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense


# Build the ANN model
model = Sequential()
model.add(Embedding(max_words, 64, input_length=max_sequence_length))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))  # Three neurons for three classes

model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use categorical_crossentropy for multi-class classification
              metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=32)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test accuracy: {test_accuracy}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.4424 - loss: 1.2444
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4798 - loss: 1.1528
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6383 - loss: 1.0331
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7610 - loss: 0.7076
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8908 - loss: 0.4052
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9571 - loss: 0.2279
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9710 - loss: 0.1378
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9930 - loss: 0.0790
Epoch 9/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9876 - loss: 0.0677
Epoch 10/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9950 - loss: 0.0402
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 208ms/step - accuracy: 0.5113 - loss: 1.6339
Test accuracy: 0.49370276927948


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Get model predictions
predictions = model.predict(test_data)

# Convert probabilities to class indices
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(test_labels.values, axis=1)

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}") #import libraries for generating report


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step
Accuracy: 0.49370277078085645
Precision: 0.43889969679443364
Recall: 0.33229400623017646
F1 Score: 0.3431206321496255


In [9]:
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
#Import Pandas
import pandas as pd
#Import numpy
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
import nltk
nltk.download('stopwords')
# %matplotlib inline  # This line can cause issues in some environments, remove if necessary

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
!pip install simpletransformers

In [11]:
from simpletransformers.classification import ClassificationModel

model1=ClassificationModel('albert','albert-base-v1',num_labels=4,use_cuda=True,args={
        "reprocess_input_data" : True,
        "use_cached_eval_features":False,
        "overwrite_output_dir": True,
        "num_train_epochs": 3 }) #Increase for better performance

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [12]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import re
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    rem_tag = re.sub(r'@\S+', '',rem_num)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_tag)
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    return " ".join(filtered_words)

combined_df['text']=combined_df['text'].map(lambda s:preprocess(s))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
from sklearn.preprocessing import LabelEncoder
train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42)

# # Create train_df and test_df from the earlier train/test splits
# train_df = pd.DataFrame({'text': tokenizer.sequences_to_texts(train_data.tolist()), 'label': train_labels.idxmax(axis=1)})
# test_df = pd.DataFrame({'text': tokenizer.sequences_to_texts(test_data.tolist()), 'label': test_labels.idxmax(axis=1)})

# Rename columns
train_df = train_df.rename(columns={'text': 'text', 'label': 'labels'})
test_df = test_df.rename(columns={'text': 'text', 'label': 'labels'})

# Encode string labels to integers
label_encoder = LabelEncoder()
train_df['labels'] = label_encoder.fit_transform(train_df['labels'])
test_df['labels'] = label_encoder.transform(test_df['labels'])  # use same encoder

train_df = train_df.sort_values(by='labels').reset_index(drop=True)

# Double-check
print(train_df.head())
print(train_df['labels'].unique())  # should show ints like [0, 1, 2, 3]

       index                                               text  labels
0  17531.png        gay okay pray proud gay hate christian hold       0
1  10446.png  laberty hangout liberty hangout ibertyhan rath...       0
2  17110.png                                            outlive       0
3  15087.png  think supporting lgbt rights making horrible p...       0
4  10745.png  aborting girls surrogate finally boy one thing...       0
[0 1 2 3]


In [14]:
model1.train_model(train_df)


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 3:   0%|          | 0/199 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Running Epoch 2 of 3:   0%|          | 0/199 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/199 [00:00<?, ?it/s]

(597, 1.1123131092269616)

In [16]:
# ------------------------------
# Inference on Eval Dataset
# ------------------------------
# Load evaluation dataset
file_path = '/content/drive/MyDrive/meme_classifications/test_ocr_text/STask-B(index,text)test.csv'
eval_df = pd.read_csv(file_path)



In [17]:
# Preprocess the text
eval_df['text'] = eval_df['text'].map(lambda s: preprocess(s))

In [18]:
# Predict using your trained model
predictions_eval, raw_outputs = model1.predict(eval_df['text'].tolist())
print(predictions_eval)

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


[2 0 1 2 2 2 0 2 2 2 2 0 3 0 2 2 0 2 1 2 3 2 0 1 2 2 3 0 2 0 3 2 0 2 0 1 0
 0 2 2 0 2 2 2 2 2 1 2 2 2 2 0 2 1 2 2 2 2 2 2 2 0 3 3 3 2 2 2 3 2 2 0 2 2
 2 2 0 2 2 0 0 2 2 1 3 2 0 2 2 3 2 2 2 0 2 2 3 0 3 0 1 3 2 2 0 2 0 2 3 2 2
 3 2 2 2 0 0 2 2 3 2 2 1 2 2 0 3 2 2 0 2 0 2 2 0 2 2 2 2 0 0 2 0 2 2 2 2 2
 1 2 0 2 2 2 2 0 0 2 0 2 2 1 0 2 2 3 0 2 2 2 2 3 2 1 2 0 2 2 2 0 2 2 1 3 0
 2 2 2 2 2 2 2 3 2 2 1 2 0 2 1 2 2 2 2 3 2 2 2 2 2 3 2 3 2 2 0 2 0 2 1 3 0
 0 2 2 2 2 2 2 0 0 0 2 2 2 2 3 2 2 2 2 2 2 2 1 1 2 3 1]


In [19]:
# Assign predictions to dataframe
eval_df['prediction'] = predictions_eval

In [20]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

if 'labels' in eval_df.columns:
    y_true = eval_df['labels'].values
    y_pred = eval_df['prediction'].values

    # Accuracy and classification report
    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=['Undirected', 'Individual', 'Community', 'Organization'], digits=4)

    # Extract precision, recall, F1
    lines = report.split('\n')
    precision = float(lines[-2].split()[2])
    recall = float(lines[-2].split()[3])
    f1_score = float(lines[-2].split()[4])

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1_score)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    class_labels = ['Undirected', 'Individual', 'Community', 'Organization']
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=class_labels, yticklabels=class_labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

In [21]:
print("Length of testdf:", len(eval_df))
print("Length of predictions_test:", len(predictions_eval))



Length of testdf: 249
Length of predictions_test: 249


In [22]:
import json
# Create submission list (keep original .png filenames)
submission_data = [{"index": row['index'], "prediction": int(row['prediction'])} for _, row in eval_df.iterrows()]


In [23]:
# Save to JSON
save_path = '/content/drive/MyDrive/submissionss.json'
with open(save_path, 'w') as json_file:
    json.dump(submission_data, json_file, indent=2)

print(f"✅ Submission file saved to: {save_path}")

✅ Submission file saved to: /content/drive/MyDrive/submissionss.json
